In [88]:
! pip install -U accelerate

^C


In [ ]:
! pip install -U transformers

     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.7 kB ? eta -:--:--
     -------------------------------------- 43.7/43.7 kB 428.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.4 MB 15.9 MB/s eta 0:00:01
   ------------ --------------------------- 3.0/9.4 MB 38.2 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.4 MB 48.0 MB/s eta 0:00:01
   -------------------------------------- - 9.0/9.4 MB 52.1 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 50.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/417.5 kB ? eta -:--:--
   ---------------------------------------- 417.5/417.5 kB ? eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 0.23.1
    Uninstalling huggingface_hub-0.23.1:
      Successfully uninstalled huggingfa

In [1]:
import pandas as pd
from datasets import Dataset
import pyarrow as pa

data_table=Dataset.from_file("./synthetic-romantic-characters-train.arrow")
data_table.to_csv("synthetic-romantic-characters.csv")

dataf=data_table.to_dict()


Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

In [2]:
def data_to_str_df(data):
    data_temp=data['conversation']
    data_t=[]
    for i in range(len(data_temp)):
        for j in range(len(data_temp[i])):
            data_t.append(data_temp[i][j]['content'])
    return data_t

datax=data_to_str_df(dataf)
datax

["*Elijah walks up to you, holding a tray of freshly baked muffins.* Hey there! Care for a muffin? They're fresh out of the oven and they're my new favorite recipe! *He grins eagerly and offers you a muffin.* And if you're interested, I can give you the recipe too! I love sharing my baking tips and recipes with others, it's just so much fun!",
 "*I take a muffin, my nose already starting to twitch from the smell.* Thanks, I'd love to have one. And if you don't mind, I'd love to get a recipe from you sometime. I'm always looking to try new baked goods.",
 "*Elijah's face lights up with excitement.* Of course, I'd be happy to share my recipe with you! I'll even write it down right now for you. I love sharing my recipes with others, it's just so much fun! *He pulls out a small notebook from his pocket and begins to write out the recipe for you.* I'll make a note to include any substitutions or adjustments for high altitude baking.",
 "*I take a bite of the muffin, savoring the sweet flavo

In [3]:
import re

def remove_annotation(dataframe_table):
    count=0
    for text in dataframe_table:
        count_star=0
        array=''
        for i in text:
            if i == '*':
                count_star+=1
            elif count_star==0:
                array+=i
            if count_star==2:
                count_star=0
        dataframe_table[count]=array
        count+=1
    return dataframe_table

removed_dataf=remove_annotation(datax)
removed_datax=pd.DataFrame(removed_dataf)
removed_datax.to_csv('annotation_removed_data.csv')

In [8]:
def slicing_sentence(dataframe_table):
    temp_array=[]
    for text in dataframe_table:
        temp=re.split(r'([!.?])',text)
        count=0
        temp_str=''
        for i in temp:
            count+=1
            if count==2:
                temp_str=temp_str.replace('[Your Name]','Anni')
                temp_array.append(temp_str+i)
                temp_str=''
                count=0
            else:
                temp_str+=i
        
    return temp_array
    

In [9]:
import re
import json
from sklearn.model_selection import train_test_split
sliced_array=slicing_sentence(removed_dataf)
def build_text_file(dataframe_table,dest_path):
    data=''
    for sentence in sliced_array:
        data+=sentence+" "
    
    data=re.sub(' +',' ',data)
    f=open(dest_path,'w')
    f.write(data)

train,test=train_test_split(sliced_array,test_size=0.15)

print(sliced_array[0])
build_text_file(train,'train_dataset.txt')
build_text_file(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


 Hey there!
Train dataset length: 99138
Test dataset length: 17495


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [11]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

c:\Users\dzvgi\anaconda3\envs\pytorch\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [12]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("openai-community/gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

c:\Users\dzvgi\anaconda3\envs\pytorch\Lib\site-packages\transformers\models\auto\modeling_auto.py:1739: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [13]:
trainer.train()  

  0%|          | 0/1047 [00:00<?, ?it/s]

c:\Users\dzvgi\anaconda3\envs\pytorch\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 2.3717, 'grad_norm': 3.760350227355957, 'learning_rate': 5e-05, 'epoch': 1.43}
{'loss': 1.9922, 'grad_norm': 3.0342743396759033, 'learning_rate': 4.296160877513712e-06, 'epoch': 2.87}
{'train_runtime': 1993.6636, 'train_samples_per_second': 16.783, 'train_steps_per_second': 0.525, 'train_loss': 2.1710499906494376, 'epoch': 3.0}


TrainOutput(global_step=1047, training_loss=2.1710499906494376, metrics={'train_runtime': 1993.6636, 'train_samples_per_second': 16.783, 'train_steps_per_second': 0.525, 'total_flos': 2185642524672000.0, 'train_loss': 2.1710499906494376, 'epoch': 3.0})

In [14]:
trainer.save_model()

In [15]:

from transformers import pipeline

gen = pipeline('text-generation',model='./gpt2', tokenizer='openai-community/gpt2')

#result = chef('Zuerst Hähnchen')[0]['generated_text']


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [16]:
gen('hello,I\'m Jake let\'s eat lunch together')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "hello,I'm Jake let's eat lunch together tomorrow. Hehe, I was wondering if you'd like to hear a story about what I've been up to lately. It happened to me when I went to this coffee shop yesterday. The coffee"}]